In [1]:
from pyspark.sql import SparkSession

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Jupyter-Spark") \
    .master("local") \
    .getOrCreate()

# Пример DataFrame
data = [("Alice", 25), ("Bob", 30)]
df = spark.createDataFrame(data, ["Name", "Age"])

# Показываем данные
df.show()

# Останавливаем SparkSession
spark.stop()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
+-----+---+



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# 1. Создаем SparkSession
spark = SparkSession.builder \
    .appName("Spark SQL with PostgreSQL") \
    .config("spark.jars", "postgresql-42.6.0.jar") \
    .getOrCreate()

# 2. Создаем DataFrame
data = [
    ("Alice", "Sales", 5000),
    ("Bob", "Sales", 6000),
    ("Charlie", "HR", 4000),
    ("David", "HR", 4500),
    ("Eve", "Sales", 5500)
]
columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data, columns)

# Показываем исходный DataFrame
print("Исходные данные:")
display(df.toPandas())

# 3. Записываем данные в PostgreSQL
jdbc_url = "jdbc:postgresql://postgres:5432/spark_db"
properties = {
    "user": "spark_user",
    "password": "spark_password",
    "driver": "org.postgresql.Driver"
}

# Записываем DataFrame в таблицу 'employees'
df.write.jdbc(url=jdbc_url, table="employees", mode="overwrite", properties=properties)

# 4. Читаем данные из PostgreSQL
df_from_postgres = spark.read.jdbc(url=jdbc_url, table="employees", properties=properties)

print("Данные, прочитанные из PostgreSQL:")
display(df_from_postgres.toPandas())

# 5. Выполняем агрегацию данных
# Группируем по департаменту и считаем общую зарплату
aggregated_df = df_from_postgres.groupBy("Department").agg(sum("Salary").alias("Total_Salary"))

print("Агрегированные данные (сумма зарплат по департаментам):")
pandas_df = aggregated_df.toPandas()
display(pandas_df)

# 6. Останавливаем SparkSession
spark.stop()

Исходные данные:


,Name,Department,Salary
0,Alice,Sales,5000
1,Bob,Sales,6000
2,Charlie,HR,4000
3,David,HR,4500
4,Eve,Sales,5500


Данные, прочитанные из PostgreSQL:


,Name,Department,Salary
0,David,HR,4500
1,Alice,Sales,5000
2,Bob,Sales,6000
3,Eve,Sales,5500
4,Charlie,HR,4000


Агрегированные данные (сумма зарплат по департаментам):


,Department,Total_Salary
0,Sales,16500
1,HR,8500


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# 1. Создаем SparkSession
spark = SparkSession.builder \
    .appName("Spark SQL with PostgreSQL and ClickHouse") \
    .config("spark.jars", "postgresql-42.6.0.jar,clickhouse-jdbc-0.4.6.jar") \
    .getOrCreate()

# 2. Создаем DataFrame
data = [
    ("Alice", "Sales", 5000),
    ("Bob", "Sales", 6000),
    ("Charlie", "HR", 4000),
    ("David", "HR", 4500),
    ("Eve", "Sales", 5500)
]
columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data, columns)

# Показываем исходный DataFrame
print("Исходные данные:")
display(df.toPandas())

# 3. Записываем данные в PostgreSQL
pg_jdbc_url = "jdbc:postgresql://postgres:5432/spark_db"
properties = {
    "user": "spark_user",
    "password": "spark_password",
    "driver": "org.postgresql.Driver"
}

# Записываем DataFrame в таблицу 'employees'
df.write.jdbc(url=pg_jdbc_url, table="employees", mode="overwrite", properties=properties)

# 4. Читаем данные из PostgreSQL
df_from_postgres = spark.read.jdbc(url=pg_jdbc_url, table="employees", properties=properties)

print("Данные, прочитанные из PostgreSQL:")
display(df_from_postgres.toPandas())

# 5. Выполняем агрегацию данных
# Группируем по департаменту и считаем общую зарплату
aggregated_df = df_from_postgres.groupBy("Department").agg(sum("Salary").alias("Total_Salary"))

print("Агрегированные данные (сумма зарплат по департаментам):")
pandas_df = aggregated_df.toPandas()
display(pandas_df)

# Записываем данные в ClickHouse через JDBC
ch_jdbc_url = "jdbc:clickhouse://clickhouse:8123/default"
properties = {
    "driver": "com.clickhouse.jdbc.ClickHouseDriver",
    "user": "custom_user",
    "password": "custom_password"
}

# Записываем данные в ClickHouse
aggregated_df.write.jdbc(url=ch_jdbc_url, table="report_ch", mode="append", properties=properties)

# Читаем данные из ClickHouse
ch_df = spark.read.jdbc(url=ch_jdbc_url, table="report_ch", properties=properties)

# Создаем временное view
ch_df.createOrReplaceTempView("report_ch")

ch_pandas_df = spark.sql("select Total_Salary from report_ch where Department = 'HR' limit 1").toPandas()

print("Агрегированные данные (сумма зарплат по департаментам) из ClickHouse где выбран департамент HR:")
display(ch_pandas_df)

# 6. Останавливаем SparkSession
spark.stop()

Исходные данные:


,Name,Department,Salary
0,Alice,Sales,5000
1,Bob,Sales,6000
2,Charlie,HR,4000
3,David,HR,4500
4,Eve,Sales,5500


Данные, прочитанные из PostgreSQL:


,Name,Department,Salary
0,Charlie,HR,4000
1,David,HR,4500
2,Eve,Sales,5500
3,Alice,Sales,5000
4,Bob,Sales,6000


Агрегированные данные (сумма зарплат по департаментам):


,Department,Total_Salary
0,Sales,16500
1,HR,8500


Агрегированные данные (сумма зарплат по департаментам) из ClickHouse где выбран департамент HR:


,Total_Salary
0,8500
